In [4]:
import conllu
import pandas as pd

In [10]:
splits = ['train', 'dev', 'test']

for split in splits:
    with open(f"it_twittiro-ud-{split}.conllu", "r", encoding="utf-8") as f:
        data = conllu.parse(f.read())
    
    # Estrai i testi e conta i token
    texts = []
    token_counts = []

    for i, sentence in enumerate(data):
        # Salva il testo se presente nei metadata
        if sentence.metadata and 'text' in sentence.metadata:
            texts.append(sentence.metadata['text'])

        # Conta i token (ignora eventuali elementi non-token come empty nodes)
        token_count = len([token for token in sentence if isinstance(token['id'], int)])
        token_counts.append(token_count)

    # Calcola la media
    average_tokens = sum(token_counts) / len(token_counts)
    print(f"{split} set:")
    print(f"Numero medio di token per frase: {average_tokens:.2f}")
    print(f"Numero totale di frasi: {len(data)}")
    print('\n')

train set:
Numero medio di token per frase: 20.77
Numero totale di frasi: 1138


dev set:
Numero medio di token per frase: 20.80
Numero totale di frasi: 144


test set:
Numero medio di token per frase: 20.96
Numero totale di frasi: 142


